In [1]:
# Create directory modular_scripts
#import os

#os.makedirs("modular_scripts")

In [ ]:
%%writefile modular_scripts/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader's for
image classification data.
"""
import os 

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataLoaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """Creates training and testing DataLoaders.

        Takes in a training directory and testing directory path and turns them into
        PyTorch Datasets and then into PyTorch DataLoaders.

        Args:
            train_dir: Path to training directory.
            test_dir: Path to testing directory,
            transform: torchvision transforms to perform on training and testing data
            batch_size: Number of samples per batch in each of the DataLoaders.
            num_workers: An integer for number of workers per DataLoader.

        Returns:
            A tuple of (train_dataLoader, test_dataLoader, class_names).
            Example usage:
                train_dataLoader, test_dataLoader, class_names = create_dataLoaders(
                    train_dir=path/to/train_dir,
                    test_dir=path/to/test_dir,
                    transform=some_transform,
                    batch_size=32,
                    num_workers=4
                )
    """

    # Use ImageFolder to create datasets(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into DataLoaders
    train_dataLoader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    test_dataLoader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )

    return train_dataLoader, test_dataLoader, class_names

Overwriting modular_scripts/data_setup.py


In [ ]:
%%writefile modular_scripts/model_builder.py
"""
Containes PyTorch model code to instantiate a TinyVGG model from the CNN Explainer website.
"""
import torch

from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website using PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape: An integer indicating number of input channels.
        hidden_units: An integer indicating number of hidden units between layers.
        output_shape: An integer indicating number of output units.
    """
    def __init__(self,
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride values are same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride values are same as kernel_size
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*3721,
                      out_features=output_shape)
        )
    
    def forward(self, x):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Writing modular_scripts/model_builder.py


In [ ]:
%%writefile modular_scripts/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

# Defining train_step
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device
) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and runs it
    through all the required training steps (forward pass,
    loss calculation, optimizer step).

    Args:
        model: A PyTorch model to be trained.
        dataLoader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on.

    Returns:
        A tuple of training loss and training accuracy metrics,
        in the form (train_loss, train_accuracy).
        Example:

        (0.1123, 0.8755)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """Test a PyTorch model for a single epoch.
    
    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.
    
    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy). 
        Example:
        (0.0223, 0.8985)
    """

    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        loss_fn: A PyTorch loss function to calculate loss on both datasets.
        epochs: An integer indicating how many epochs to train for.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for 
        each epoch.
        In the form: {train_loss: [...],
                    train_acc: [...],
                    test_loss: [...],
                    test_acc: [...]} 
        For example if training for epochs=2: 
                    {train_loss: [2.0616, 1.0537],
                    train_acc: [0.3945, 0.3945],
                    test_loss: [1.2641, 1.5706],
                    test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | " 
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Overwriting modular_scripts/engine.py


In [20]:
%%writefile modular_scripts/utils.py
"""
Contains various utility functions for PyTorch model training and saving
"""
import torch, os, sys, requests, zipfile, gdown, torchvision
from pathlib import Path
from tqdm import tqdm
from PIL import Image
from typing import List, Tuple
from torchvision import transforms
import matplotlib.pyplot as plt
import shutil

try:
    import splitfolders
except ModuleNotFoundError:
    import subprocess
    import sys

    print("[INFO] splitfolders module not found. Installing...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'split-folders[full]'])
    import splitfolders


# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

def save_model(
    model: torch.nn.Module,
    target_dir: str,
    model_name: str):
    """Saves a PyTorch model to a target directory
    
    Args:
        model: A target PyTorch model to save
        target_dir: A directory to save the model
        model_name: A filename for the saved model. Should include 
        either .pth ot .pt as the file extension
    
    Example usage:
        save_model(
            model=model_0,
            target_dir="models",
            model_name="rubbish_classifier.pth"ArithmeticError
        )
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)

def split_data(input_folder: str, output_folder: str, ratio: tuple = (.8, .1, .1)):
    """Split data into train, val and test datasets using split-folders: https://github.com/jfilter/split-folders/tree/main

    Args:
        input_folder: path of the folder with the dataset with the following format:
                    input/
                        class1/
                            img1.jpg
                            img2.jpg
                            ...
                        class2/
                            imgWhatever.jpg
                            ...
                        ...
        output_folder: path to save the datasets as follows:
                    output/
                        train/
                            class1/
                                img1.jpg
                                ...
                            class2/
                                imga.jpg
                                ...
                        val/
                            class1/
                                img2.jpg
                                ...
                            class2/
                                imgb.jpg
                                ...
                        test/
                            class1/
                                img3.jpg
                                ...
                            class2/
                                imgc.jpg
                                ...
        ratio: tuple to specify the ratio to split into train, val and test.
    
    Example of use:
        split_data(input_folder='data/rubbish_dataset', output_folder=data/rubbish_dataset, ratio=(.8, .1, .1))
    """
    # Setup directory path
    train_dir = output_folder / "train"
    test_dir = output_folder / "test"
    val_dir = output_folder / "val"

    splitfolders.ratio(input_folder, output=output_folder,
                        seed=1337, ratio=ratio, group_prefix=None, move=False) # default values
    shutil.rmtree(input_folder)

    return train_dir, test_dir, val_dir
    
def bulk_image_convertor(dataset_path: str, format: str ="jpg"):
    """Converts Images from the labels folders of a given dataset folder into a given format.
    
    Args:
        path: String path of dataset folder
        format: Format to convert to
    
    Example of use:
        # cans class convertion
        bulk_image_convertor(path="data/dataset",
                        format="jpg")
    """
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)

        # Check if the current item in the dataset path is a directory
        if os.path.isdir(label_path):
            # Call image_convertor for the current label folder
            image_convertor(label_path, format=format)

def image_convertor(path: str, format: str = 'jpg'):
    """Converts Images from a given folder into a given format.
    
    Args:
        path: String path of the images to convert
        format: Format to convert to
    
    Example of use:
        # cans class convertion
        image_convertor(path="data/dataset/cans/",
                        format="jpg")
    """
    count=0
    path=Path(path)
    for file in tqdm(path.glob("./*")):
        f, e = os.path.splitext(file)
        renameFile = f + "."+format.lower()
        if e.lower() != "."+format.lower():
            old_file=file
            count+=1
            try:
                with Image.open(file) as img:
                    img.save(renameFile)
            except OSError:
                print("cannot convert", file)
            os.remove(old_file)
    print(f"{count} images converted to '{format}' in '{path}'")

def download_data(
    source: str,
    destination: str,
    from_gdrive: bool = False,
    remove_source: bool = True) -> Path:
    """Download and extract the data of a Zip file from am external source like GitHub or a Gooogle Drive folder.
    
    Args:
        source (str): A link to a zipped file containing data. In case of a Gooogle Drive, provide only the file ID
        destination (str): name of the folder where you want to save the data
        remove_source: boolean to remove source file after downloading
    """
    
    # Setup a path to a data folder
    data_path = Path("data/")
    images_path = data_path / destination

    # If the data folder doesn't exist, download it and prepare it.
    if images_path.is_dir():
        print(f"'{images_path}' directory already exists, skipping directory creation...")
    else:
        print(f"'{images_path}' does not exist, creating directory...")
        images_path.mkdir(parents=True, exist_ok=True)

    target_file = Path(source).name

    if from_gdrive:
        url = 'https://drive.google.com/uc?id='+ source
        output = str(data_path) + '/' + str(target_file)
        print(f"[INFO] Donwloading {target_file} from https://drive.google.com/uc?id={source}")
        gdown.download(url, output, quiet=False)
    else:
        with open(data_path / target_file, 'wb') as f:
            res = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(res.content)

    # Unzip data
    with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
        print(f"[INFO] Unzipping {target_file} data...")
        zip_ref.extractall(images_path)
    
    if remove_source:
        os.remove(data_path / target_file)

    return images_path

def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str],
                        image_size: Tuple[int, int] = (224, 224),
                        transform: torchvision.transforms = None,
                        device: torch.device = device):
    """Make a prediction and plot an image using a given model.
    
    Args:
        mode: Model to make the prediction
        image_path: Path of the image on which the prediction will occur
        class_names: Names of the classess to predict
        transform: Transforms to apply on the image to predict. Good practices to use same as the used on the pretrained model.
        device: Target device
    
    Example of use:
        pred_and_plot_image(model=model, 
                            image_path=image_path,
                            class_names=class_names,
                            transform=weights.transforms(), optionally pass in a specified transform from our pretrained model weights
                            image_size=(224, 224))
    """
    # Open Image
    img = Image.open(image_path)

    # Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),

        ])
    
    ### Predict on image ### 
    # Make sure mode is on the target device
    model.to(device)

    # Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # Transform and add an extra dimesion to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(img).unsqueeze(dim=0)
        # Make prediction on image with extra dimension and send it ot the target device
        target_image_pred = model(transformed_image.to(device))
    
    # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # Plot image with predicted label and probability
    plt.figure()
    plt.imshow(img)
    plt.title(f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}")
    plt.axis(False)

Overwriting modular_scripts/utils.py


In [ ]:
%%writefile modular_scripts/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

# Start timer
from timeit import default_timer as timer 

from torchvision import transforms

# Set up hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = ""
test_dir = ""

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create DataLoaders using data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

start_time = timer()

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device) 

# End timer and print total time
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds.")

# Save the model
utils.save_model(model=model,
                 target_dir="models",
                 model_name="rubbish_classifier_tinyvgg_model.pth")

Writing modular_scripts/train.py


In [2]:
from datasets import load_dataset

datasetOne = load_dataset("edwinpalegre/trashnet_enhanced")
datasetTwo = load_dataset("ethanwan/trash_classification")

datasetOne, datasetTwo

c:\Users\juanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 68/68 [00:02<00:00, 33.34it/s]


(DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 19892
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 456
     })
     validation: Dataset({
         features: ['image', 'label'],
         num_rows: 133
     })
     test: Dataset({
         features: ['image', 'label'],
         num_rows: 68
     })
 }))

In [3]:
datasetOne_Train = datasetOne['train']['image']
datasetTwo_Train = datasetTwo['train']['image']
datasetTwo_Val = datasetTwo['validation']['image']
datasetTwo_Test = datasetTwo['test']['image']

MemoryError: 

In [10]:
import os, random
from PIL import Image
import io

# Assuming your dataset has a 'label' field
datasetOne_trainLabels = datasetOne['train']['label']
datasetTwo_trainLabels = datasetTwo['train']['label']
datasetTwo_valLabels = datasetTwo['validation']['label']
datasetTwo_testLabels = datasetTwo['test']['label']

# Create a directory for each label
for label in set(datasetOne_trainLabels):
    os.makedirs(f'datasetOne_trainImages/{label}', exist_ok=True)

# Create a directory for each label
for label in set(datasetTwo_trainLabels):
    os.makedirs(f'datasetTwo_trainImages/{label}', exist_ok=True)

# Create a directory for each label
for label in set(datasetTwo_valLabels):
    os.makedirs(f'datasetTwo_valImages/{label}', exist_ok=True)

# Create a directory for each label
for label in set(datasetTwo_testLabels):
    os.makedirs(f'datasetTwo_testImages/{label}', exist_ok=True)

# Save each image in its corresponding label folder
for i, (image, label) in enumerate(zip(datasetOne_Train, datasetOne_trainLabels)):
    with io.BytesIO() as output:
        image.save(output, format="PNG")  # Adjust format if needed
        image_bytes = output.getvalue()

    with open(f'datasetOne_trainImages/{label}/image_{random.randint(i)}.png', 'wb') as file:
        file.write(image_bytes)

# Save each image in its corresponding label folder
for i, (image, label) in enumerate(zip(datasetTwo_Train, datasetTwo_trainLabels)):
    with io.BytesIO() as output:
        image.save(output, format="PNG")  # Adjust format if needed
        image_bytes = output.getvalue()

    with open(f'datasetTwo_trainImages/{label}/image_{random.randint(i)}.png', 'wb') as file:
        file.write(image_bytes)

# Save each image in its corresponding label folder
for i, (image, label) in enumerate(zip(datasetTwo_Val, datasetTwo_valLabels)):
    with io.BytesIO() as output:
        image.save(output, format="PNG")  # Adjust format if needed
        image_bytes = output.getvalue()

    with open(f'datasetTwo_valImages/{label}/image_{random.randint(i)}.png', 'wb') as file:
        file.write(image_bytes)

# Save each image in its corresponding label folder
for i, (image, label) in enumerate(zip(datasetTwo_Test, datasetTwo_testLabels)):
    with io.BytesIO() as output:
        image.save(output, format="PNG")  # Adjust format if needed
        image_bytes = output.getvalue()

    with open(f'datasetTwo_testImages/{label}/image_{random.randint(i)}.png', 'wb') as file:
        file.write(image_bytes)
